# Extreme Natural events: Cyclones

## Data Extraction from EONET NASA API

In [2]:
import pandas as pd
import numpy as np
import re
import math
import requests
import json
import matplotlib
from pandas import ExcelFile
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.express
import reverse_geocoder as rg
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


The natural events in the EONET's API are divided in two primarily groups according with their status:

* ```'status' = 'open'``` means the natural event has not ended.

* ```'status' = 'closed'``` means the natural event has finished. When this happen we will have as well an aditional field labeled as closed.

## 1. Data extraction and wrangling

### Extracting data from the EONET's API

In [3]:
#For extracting the category of each natural event

def extract_cat(lista):
    """
    Takes a list of dictionaries and return the value of the first element
    of the list for the key 'title'
    """
    return lista[0]["title"]


# Extracts the date of each natural event
def extract_date(lista):
    """
    Takes a list of dictionaries and return the value of the first element
    of the list for the key 'date'
    """
    return lista[0]["date"]

# Extracts the type of Geojson coordinates of each natural event

def extract_geom(lista):
    """
    Takes a list of dictionaries and removes the key 'date' for the first 
    dictionary
    """
    del lista[0]["date"]
    return lista[0]
    
base = "https://eonet.sci.gsfc.nasa.gov/api/v2.1/events?status=" 

def get_events(st):
    
    """
    Arguments: "open" or "closed"
    Returns a the data set of the events with status "st" 
    which happened since 1980-01-10 until teh current date.
    
    """
    url = base + f"{st}"
    result = requests.get(url)
    
    df_0 = pd.DataFrame(result.json()["events"], columns = ["id", "title", "categories", "description", "sources", "geometries", "closed"])
    df_0.categories = df_0.categories.apply(extract_cat) # Extract category from the original dictionary
    df_0["date"] = df_0.geometries.apply(extract_date)
    df_0["geometries"] = df_0.geometries.apply(extract_geom)
    df_0 = df_0.drop(columns = ["sources"])
    return df_0[['id', 'title', 'categories', 'description', 'date', 'closed','geometries']]


In [4]:
df_op = get_events("open")
df_cl = get_events("closed")

In [5]:
df_ev = (pd
        .concat([df_op, df_cl], ignore_index=True)
        .sort_values("date", ascending = False)
        .reset_index(drop=True))

df_ev.dtypes

id             object
title          object
categories     object
description    object
date           object
closed         object
geometries     object
dtype: object

In [6]:
df_ev.head()

,id,title,categories,description,date,closed,geometries
0,EONET_4617,"Wildfire - Molina, Chile",Wildfires,,2020-03-16T17:18:00Z,NaN,"{'type': 'Point', 'coordinates': [-71.28198, -..."
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13T00:00:00Z,NaN,"{'type': 'Point', 'coordinates': [52.4, -14.3]}"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13T00:00:00Z,NaN,"{'type': 'Point', 'coordinates': [153.8, -16.9]}"
3,EONET_4614,"Wildfire - Constitucion, Chile",Wildfires,,2020-03-11T15:37:00Z,NaN,"{'type': 'Point', 'coordinates': [-72.42013, -..."
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11T00:00:00Z,2020-03-16T00:00:00Z,"{'type': 'Point', 'coordinates': [116.9, -16.4]}"


In [12]:
df_ev.categories.value_counts()

Severe Storms           1825
Wildfires               1535
Volcanoes                864
Floods                   116
Sea and Lake Ice          60
Water Color               17
Dust and Haze             17
Earthquakes               15
Temperature Extremes      14
Drought                    7
Manmade                    5
Snow                       3
Landslides                 2
Name: categories, dtype: int64

In [8]:
df_ev["date"]= pd.to_datetime(df_ev['date'],infer_datetime_format = True).dt.date
df_ev["closed"]= pd.to_datetime(df_ev['closed'],infer_datetime_format = True).dt.date

In [9]:
df_ev["date"]= pd.to_datetime(df_ev['date'],infer_datetime_format = True)
df_ev["closed"]= pd.to_datetime(df_ev['closed'],infer_datetime_format = True)

In [10]:
df_ev.dtypes

id                     object
title                  object
categories             object
description            object
date           datetime64[ns]
closed         datetime64[ns]
geometries             object
dtype: object

In [11]:
df_ev.sort_values("date")

,id,title,categories,description,date,closed,geometries
4479,EONET_713,"Tupungatito Volcano, Chile-Argentina",Volcanoes,,1980-01-10,1980-01-11,"{'type': 'Point', 'coordinates': [-69.797, -33..."
4478,EONET_936,"Nyamuragira Volcano, DR Congo",Volcanoes,,1980-01-30,1980-02-24,"{'type': 'Point', 'coordinates': [29.2, -1.408]}"
4477,EONET_533,"Krakatau Volcano, Indonesia",Volcanoes,,1980-03-15,1980-12-31,"{'type': 'Point', 'coordinates': [105.423, -6...."
4476,EONET_1043,"St. Helens Volcano, Washington",Volcanoes,,1980-03-27,1986-10-28,"{'type': 'Point', 'coordinates': [-122.18, 46.2]}"
4475,EONET_886,"Lopevi Volcano, Vanuatu",Volcanoes,,1980-04-15,1980-08-20,"{'type': 'Point', 'coordinates': [168.346, -16..."
...,...,...,...,...,...,...,...
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [116.9, -16.4]}"
3,EONET_4614,"Wildfire - Constitucion, Chile",Wildfires,,2020-03-11,NaT,"{'type': 'Point', 'coordinates': [-72.42013, -..."
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [153.8, -16.9]}"
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [52.4, -14.3]}"


In [209]:
storms = df_ev[df_ev.categories== "Severe Storms"].copy()

In [210]:
storms.head()

,id,title,categories,description,date,closed,geometries
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}"


In [211]:
len(storms)

1825

In [212]:
storms["coordinates"]= storms.loc[:, ("geometries")].apply(lambda x: x["coordinates"])

In [213]:
storms

,id,title,categories,description,date,closed,geometries,coordinates
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","[-14.3, 52.4]"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","[-16.9, 153.8]"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","[-16.4, 116.9]"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","[-13.4, 118.7]"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","[-15.6, 139.5]"
...,...,...,...,...,...,...,...,...
4178,EONET_2351,Typhoon #34,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","[14.7, 131.6]"
4179,EONET_2312,Cyclone #4,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","[-10.2, 65.4]"
4180,EONET_2409,Cyclone #2,Severe Storms,,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","[-13.3, 119.4]"
4181,EONET_2410,Tropical Storm #1,Severe Storms,,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","[-9.4, 97.4]"


In [189]:
def rev(x):
    y = x[::-1]
    return y

In [190]:
storms["coordinates"] = storms.coordinates.apply(rev)

In [214]:
storms.head()

,id,title,categories,description,date,closed,geometries,coordinates
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","[-14.3, 52.4]"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","[-16.9, 153.8]"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","[-16.4, 116.9]"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","[-13.4, 118.7]"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","[-15.6, 139.5]"


In [215]:
storms["list_length"] = storms.coordinates.apply(lambda x: len(x)).copy()

In [216]:
storms = storms[~(storms.list_length < 2)]

In [217]:
storms.drop(columns = "list_length", inplace= True)

In [218]:
storms

,id,title,categories,description,date,closed,geometries,coordinates
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","[-14.3, 52.4]"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","[-16.9, 153.8]"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","[-16.4, 116.9]"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","[-13.4, 118.7]"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","[-15.6, 139.5]"
...,...,...,...,...,...,...,...,...
4178,EONET_2351,Typhoon #34,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","[14.7, 131.6]"
4179,EONET_2312,Cyclone #4,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","[-10.2, 65.4]"
4180,EONET_2409,Cyclone #2,Severe Storms,,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","[-13.3, 119.4]"
4181,EONET_2410,Tropical Storm #1,Severe Storms,,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","[-9.4, 97.4]"


In [219]:
storms["coordinates"]= storms.coordinates.apply(lambda x: (x[0], x[1])).copy()

/Users/Fabi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



As observed, we have geometries but explicit cities or countries in which the extreme events took place. For that reason we exract the latitud and longitud from ``geometries`` in order to use ``reverse_geocoder`` module:

,id,title,categories,description,date,closed,geometries,coordinates
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)"
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)"
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)"
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)"
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)"


In [222]:
storms["location"]= storms.coordinates.apply(lambda x: rg.search(x)[0]["name"])
storms["admin"]= storms.coordinates.apply(lambda x: rg.search(x)[0]["admin1"])
storms["code"]= storms.coordinates.apply(lambda x: rg.search(x)[0]["cc"])


/Users/Fabi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/Fabi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/Fabi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [224]:
storms

,id,title,categories,description,date,closed,geometries,coordinates,location,admin,code
1,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)",Antalaha,Sava,MG
2,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)",Proserpine,Queensland,AU
4,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)",Dampier,Western Australia,AU
19,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID
20,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)",Alyangula,Northern Territory,AU
...,...,...,...,...,...,...,...,...,...,...,...
4178,EONET_2351,Typhoon #34,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","(14.7, 131.6)",San Policarpo,Eastern Visayas,PH
4179,EONET_2312,Cyclone #4,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","(-10.2, 65.4)",Cargados Carajos,Cargados Carajos,MU
4180,EONET_2409,Cyclone #2,Severe Storms,,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","(-13.3, 119.4)",Lailunggi,East Nusa Tenggara,ID
4181,EONET_2410,Tropical Storm #1,Severe Storms,,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","(-9.4, 97.4)",West Island,,CC


In [225]:
codes= pd.read_excel("/Users/Fabi/Documents/GitHub/wfpdisrel/country_codes.xlsx")

In [226]:
codes.head()

,Country,ISO2,ISO3,Numeric
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20


In [227]:
storms= storms.merge(codes[["ISO2", "ISO3", "Country"]], 
                     left_on= "code", 
                     right_on= "ISO2", 
                     how= "left").drop(columns= "code")

In [228]:
storms

,id,title,categories,description,date,closed,geometries,coordinates,location,admin,ISO2,ISO3,Country
0,EONET_4615,Tropical Cyclone Herold,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)",Antalaha,Sava,MG,MDG,Madagascar
1,EONET_4616,Tropical Cyclone Gretel,Severe Storms,,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)",Proserpine,Queensland,AU,AUS,Australia
2,EONET_4612,Tropical Cyclone 21S,Severe Storms,,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)",Dampier,Western Australia,AU,AUS,Australia
3,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
4,EONET_4595,Tropical Cyclone Esther,Severe Storms,,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)",Alyangula,Northern Territory,AU,AUS,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814,EONET_2351,Typhoon #34,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","(14.7, 131.6)",San Policarpo,Eastern Visayas,PH,PHL,Philippines (the)
1815,EONET_2312,Cyclone #4,Severe Storms,,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","(-10.2, 65.4)",Cargados Carajos,Cargados Carajos,MU,MUS,Mauritius
1816,EONET_2409,Cyclone #2,Severe Storms,,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","(-13.3, 119.4)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
1817,EONET_2410,Tropical Storm #1,Severe Storms,,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","(-9.4, 97.4)",West Island,,CC,CCK,Cocos (Keeling) Islands (the)


In [229]:
storms.drop(columns= "description", inplace= True)

In [230]:
storms.reset_index(drop= True, inplace= True)

In [231]:
storms

,id,title,categories,date,closed,geometries,coordinates,location,admin,ISO2,ISO3,Country
0,EONET_4615,Tropical Cyclone Herold,Severe Storms,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-14.3, 52.4]}","(-14.3, 52.4)",Antalaha,Sava,MG,MDG,Madagascar
1,EONET_4616,Tropical Cyclone Gretel,Severe Storms,2020-03-13,NaT,"{'type': 'Point', 'coordinates': [-16.9, 153.8]}","(-16.9, 153.8)",Proserpine,Queensland,AU,AUS,Australia
2,EONET_4612,Tropical Cyclone 21S,Severe Storms,2020-03-11,2020-03-16,"{'type': 'Point', 'coordinates': [-16.4, 116.9]}","(-16.4, 116.9)",Dampier,Western Australia,AU,AUS,Australia
3,EONET_4596,Tropical Cyclone Ferdinand,Severe Storms,2020-02-23,2020-03-05,"{'type': 'Point', 'coordinates': [-13.4, 118.7]}","(-13.4, 118.7)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
4,EONET_4595,Tropical Cyclone Esther,Severe Storms,2020-02-23,2020-03-04,"{'type': 'Point', 'coordinates': [-15.6, 139.5]}","(-15.6, 139.5)",Alyangula,Northern Territory,AU,AUS,Australia
...,...,...,...,...,...,...,...,...,...,...,...,...
1814,EONET_2351,Typhoon #34,Severe Storms,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [14.7, 131.6]}","(14.7, 131.6)",San Policarpo,Eastern Visayas,PH,PHL,Philippines (the)
1815,EONET_2312,Cyclone #4,Severe Storms,2000-01-01,2001-01-05,"{'type': 'Point', 'coordinates': [-10.2, 65.4]}","(-10.2, 65.4)",Cargados Carajos,Cargados Carajos,MU,MUS,Mauritius
1816,EONET_2409,Cyclone #2,Severe Storms,1999-12-10,1999-12-20,"{'type': 'Point', 'coordinates': [-13.3, 119.4]}","(-13.3, 119.4)",Lailunggi,East Nusa Tenggara,ID,IDN,Indonesia
1817,EONET_2410,Tropical Storm #1,Severe Storms,1999-12-10,1999-12-22,"{'type': 'Point', 'coordinates': [-9.4, 97.4]}","(-9.4, 97.4)",West Island,,CC,CCK,Cocos (Keeling) Islands (the)


In [232]:
storms.to_json("storms.json")